In [ ]:
from pathlib import Path
import os
if not str(Path.cwd()).endswith('/Cloud-Telescope'):
    os.chdir(Path.cwd().parent)
    print(f'New working dir is {Path.cwd()}')    
import pandas as pd
import importlib

import src.config
importlib.reload(src.config)

import src.d01_data.read_files  
importlib.reload(src.d01_data.read_files)
import src.d01_data.save_files
importlib.reload(src.d01_data.save_files)
import src.d01_data.log_names_enum
importlib.reload(src.d01_data.log_names_enum)

import src.d04_analysation.count_percent
importlib.reload(src.d04_analysation.count_percent)

import src.d02_intermediate.filter_telescope_ips
importlib.reload(src.d02_intermediate.filter_telescope_ips)

import src.d04_analysation.scanning
importlib.reload(src.d04_analysation.scanning)

import src.d04_analysation.arrival_rate
importlib.reload(src.d04_analysation.arrival_rate)

import src.d04_analysation.scnas_mapped_to_country
importlib.reload(src.d04_analysation.scnas_mapped_to_country)

import src.d04_analysation.scans_proto_country
importlib.reload(src.d04_analysation.scans_proto_country)

import src.d04_analysation.find_scanner
importlib.reload(src.d04_analysation.find_scanner)

import src.d04_analysation.scan_aggrssivity_mapped_countries
importlib.reload(src.d04_analysation.scan_aggrssivity_mapped_countries)

import src.d04_analysation.reverse_dns_lookup
importlib.reload(src.d04_analysation.reverse_dns_lookup)

import src.d04_analysation.scan_education
importlib.reload(src.d04_analysation.scan_education)

import src.d04_analysation.comparison
importlib.reload(src.d04_analysation.comparison)

import src.d03_visiualisation.tabel_visiualisation
importlib.reload(src.d03_visiualisation.tabel_visiualisation)

import src.d03_visiualisation.bar_diagram
importlib.reload(src.d03_visiualisation.bar_diagram)

import src.d03_visiualisation.head_map
importlib.reload(src.d03_visiualisation.head_map)

import src.d03_visiualisation.graf_arrival_rate
importlib.reload(src.d03_visiualisation.graf_arrival_rate)

import src.d03_visiualisation.graf_interval_threshold
importlib.reload(src.d03_visiualisation.graf_interval_threshold)

import src.d03_visiualisation.tabel_visiualisation
importlib.reload(src.d03_visiualisation.tabel_visiualisation)

### load darknet Teleskope data

In [ ]:
from src.d01_data.read_files import load_latest_dataframe
from src.d01_data.log_names_enum import LogNamesEnum
from src.config import *

address_notice: pd.DataFrame = load_latest_dataframe(LogNamesEnum.xcaddressnotice)
address_notice: pd.DataFrame = address_notice[address_notice['note'] == 'ScanUDP::Address_Scan']
address_notice_unique: pd.DataFrame = address_notice.drop_duplicates(subset='src', keep='first')
address_conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.caddressconn)

port_notice: pd.DataFrame = load_latest_dataframe(LogNamesEnum.zportnotice)
port_notice: pd.DataFrame = port_notice[port_notice['note'] == 'ScanUDP::Port_Scan']
port_notice_unique: pd.DataFrame = port_notice.drop_duplicates(subset='src', keep='first')
port_conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.cportconn)

### load cloud Teleskope data

In [ ]:
from src.config import *
down_time_start = pd.Timestamp("2025-01-25 11:00:00").timestamp()
down_time_end = pd.Timestamp("2025-01-27 19:00:00").timestamp()

port_scan: pd.DataFrame = load_latest_dataframe(LogNamesEnum.portscans)
port_scan = port_scan[(port_scan['ts'] < down_time_start) | (port_scan['ts'] > down_time_end)]
port_scan_unique: pd.DataFrame = port_scan.drop_duplicates(subset='src', keep='first')

address_scan: pd.DataFrame = load_latest_dataframe(LogNamesEnum.addressscans)
address_scan = address_scan[(address_scan['ts'] < down_time_start) | (address_scan['ts'] > down_time_end)]
address_scan_unique: pd.DataFrame = address_scan.drop_duplicates(subset='src', keep='first')


In [ ]:
conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.conn)
conn_udp: pd.DataFrame = conn[conn['proto'] == 'udp']

In [ ]:
from src.config import *

address_cloud_in_darknet: pd.DataFrame = address_scan_unique[address_scan_unique['src'].isin(address_caida)]

address_cloud_not_in_darknet: pd.DataFrame = address_scan_unique[~address_scan_unique['src'].isin(address_caida)]

In [ ]:
from src.config import *

port_cloud_in_darknet: pd.DataFrame = port_scan_unique[port_scan_unique['src'].isin(port_caida)]

port_cloud_not_in_darknet: pd.DataFrame = port_scan_unique[~port_scan_unique['src'].isin(port_caida)]

### show top scanner

In [ ]:
from src.d04_analysation.count_percent import *
from src.d03_visiualisation.bar_diagram import *
from src.d04_analysation.reverse_dns_lookup import *
def show_top_scanner(darknet_df: pd.DataFrame):
    conn_rir: pd.DataFrame = conn_udp[['id.orig_h', 'orig_as_name_rir']]
    conn_rir = conn_rir.drop_duplicates(subset='id.orig_h', keep='first')
    
    darknet_df_calculation: pd.DataFrame = column_calculation(darknet_df, {'src': -1}, False)[0].result_as_df
    darknet_df_calculation = darknet_df_calculation.drop(index=darknet_df_calculation.index[0])
    darknet_df_calculation = darknet_df_calculation.head(10) 
    
    darknet_df_calculation_rir: pd.DataFrame = darknet_df_calculation.merge(conn_rir, how='left', left_on='src', right_on='id.orig_h')
    
    
    basic_bar_diagram(df=darknet_df_calculation_rir, x_column='orig_as_name_rir', y_column='total', x_lable='RIR Name', y_lable='Anzahl Scanner')
    
show_top_scanner(cloud_darknet)

### calc caida address

In [ ]:
from src.d04_analysation.comparison import *

address_cloud_in_darknet_country = add_country_information(address_cloud_in_darknet, conn_udp)
address_cloud_not_in_darknet_country = add_country_information(address_cloud_not_in_darknet, conn_udp)

address_cloud_in_darknet_usa = address_cloud_in_darknet_country[address_cloud_in_darknet_country['orig_timezone_location'] == 'America']
address_cloud_in_darknet_not_usa = address_cloud_in_darknet_country[address_cloud_in_darknet_country['orig_timezone_location'] != 'America']

address_cloud_not_in_darknet_usa = address_cloud_not_in_darknet_country[address_cloud_not_in_darknet_country['orig_timezone_location'] == 'America']
address_cloud_not_in_darknet_not_usa = address_cloud_not_in_darknet_country[address_cloud_not_in_darknet_country['orig_timezone_location'] != 'America']

### calc caida port

In [ ]:
from src.d04_analysation.comparison import *

port_cloud_in_darknet_country = add_country_information(port_cloud_in_darknet, conn_udp)
port_cloud_not_in_darknet_country = add_country_information(port_cloud_not_in_darknet, conn_udp)

port_cloud_in_darknet_usa = port_cloud_in_darknet_country[port_cloud_in_darknet_country['orig_timezone_location'] == 'America']
port_cloud_in_darknet_not_usa = port_cloud_in_darknet_country[port_cloud_in_darknet_country['orig_timezone_location'] != 'America']

port_cloud_not_in_darknet_usa = port_cloud_not_in_darknet_country[port_cloud_not_in_darknet_country['orig_timezone_location'] == 'America']
port_cloud_not_in_darknet_not_usa = port_cloud_not_in_darknet_country[port_cloud_not_in_darknet_country['orig_timezone_location'] != 'America']

In [ ]:
display(cloud_in_darknet_usa)
display(cloud_in_darknet_not_usa)
display(cloud_not_in_darknet_usa)
display(cloud_not_in_darknet_not_usa)

### show comparison address

In [ ]:
from src.d03_visiualisation.bar_diagram import *
def show_comparison_result(comparison_result_df: pd.DataFrame, saving_name: str, legend: bool):
    
    labels = ['Gesamt', 'USA', 'Nicht USA']
    
    plot_overlapping_bars(comparison_result_df['y1_values'].tolist(), comparison_result_df['y2_values'].tolist(), labels, saving_name=saving_name, legend=legend, label_1='In CAIDA',label_2='Nicht in CAIDA', width=4, height=5, gap=0.12, bar_width=0.1).show()
    return comparison_result_df

address_comparison_result_df: pd.DataFrame = pd.DataFrame({
        'y1_values': [len(address_cloud_in_darknet_country), len(address_cloud_in_darknet_usa), len(address_cloud_in_darknet_not_usa)],
        'y2_values': [ len(address_cloud_not_in_darknet_country), len(address_cloud_not_in_darknet_usa), len(address_cloud_not_in_darknet_not_usa)]
    })
port_comparison_result_df: pd.DataFrame = pd.DataFrame({
        'y1_values': [len(port_cloud_in_darknet_country), len(port_cloud_in_darknet_usa), len(port_cloud_in_darknet_not_usa)],
        'y2_values': [ len(port_cloud_not_in_darknet_country), len(port_cloud_not_in_darknet_usa), len(port_cloud_not_in_darknet_not_usa)]
    })

show_comparison_result(address_comparison_result_df, 'caida_address_overlapping_bars', True)
show_comparison_result(port_comparison_result_df, 'caida_port_overlapping_bars', False)

In [ ]:
from src.d03_visiualisation.tabel_visiualisation import *

conn_udp_rir_ = conn_udp[['id.orig_h', 'orig_as_name_rir']]
conn_udp_rir_ = conn_udp_rir_.drop_duplicates(subset='id.orig_h', keep='first')

top_10_address_in_c = pd.DataFrame({
    'src': top_10_address_in_caida
})
top_10_address_in_c = top_10_address_in_c.merge(conn_udp_rir_, how='left', left_on='src', right_on='id.orig_h')

top_10_address_not_in_c = pd.DataFrame({
    'src': top_10_address_not_in_caida
})
top_10_address_not_in_c = top_10_address_not_in_c.merge(conn_udp_rir_, how='left', left_on='src', right_on='id.orig_h')

top_10_address = pd.DataFrame({
    'In_CAIDA_gefunden': top_10_address_in_c['src'].tolist(),
    'Nicht in_CAIDA_gefunden': top_10_address_not_in_c['src'].tolist() 
})


top_10_port_in_c = pd.DataFrame({
    'src': top_10_port_in_caida
})
top_10_port_in_c = top_10_port_in_c.merge(conn_udp_rir_, how='left', left_on='src', right_on='id.orig_h')

top_10_port_not_in_c = pd.DataFrame({
    'src': top_10_port_not_in_caida
})
top_10_port_not_in_c = top_10_port_not_in_c.merge(conn_udp_rir_, how='left', left_on='src', right_on='id.orig_h')

top_10_port = pd.DataFrame({
    'In_CAIDA_gefunden': top_10_port_in_c['src'].tolist(),
    'Nicht in_CAIDA_gefunden': top_10_port_not_in_c['src'].tolist() 
})
column_widths = {
        "In_CAIDA_gefunden": 0.7,
        "in_CAIDA_gefunden": 0.7,
    }
row_heights = [0.08, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

show_two_basic_table(top_10_address, col_widths=column_widths, row_heights=row_heights, saving_name='caida_address_rir')